# Requirements
First, we import all the packages and modules we need for our pipeline.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn_pandas import DataFrameMapper, cross_val_score
import sklearn.preprocessing

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score


# Raw Material
First, we retrieve our training and test datasets from the csv files provided by Kaggle and store them in Pandas DataFrame objects.

In [62]:
train_df = pd.read_csv('../../data/train.csv')
test_df = pd.read_csv('../../data/test.csv')

print 'train:', train_df.shape
print 'test:', test_df.shape

train: (1460, 81)
test: (1459, 80)


For later use, we store the feature labels of our training data in two variables, separated by categorical and continious features. Additionally, we store the label of our target variable (SalePrice).

In [63]:
CAT_VARS = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 
            'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
            'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
            'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
            'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
            'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
            'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
            'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
            'MoSold', 'SaleType', 'SaleCondition']
CONT_VARS = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
             'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
             'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
             'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
             'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
             'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold']

TARGET_VAR = ['SalePrice',]


# Preprocessing
The house IDs are currently saved in the datasets as ordinary columns. Pandas, however, allows for explicitely specifying indeces, i.e. row labels (much like our feature/column labels). The following makes sure we use our own IDs as indeces.

In [64]:
train_df.set_index('Id', inplace=True)
test_df.set_index('Id', inplace=True)

The data contains of both categorial and continuous values. The following makes sure Pandas knows about this distinction and does not confuse the two by explicetely defining categorial features as such.

In [65]:
for cat_var in CAT_VARS:
    train_df[cat_var].astype('category')

Some prediction models require continuous features only. Thus, dummify() converts categorial features with *m* different possible classes into *m* new features (columns) – one feature for each class. These dummified features are either 1 or 0. 

For clarification of what's happening, consider the following example:

| Id | Street | → | Street_Gravel | Street_Paved |
|----|--------|---|---------------|--------------|
| 1  | Gravel | → | 1             | 0            |
| 2  | Paved  | → | 0             | 1            |
| 3  |        | → | 0             | 0            |
| 4  | Paved  | → | 0             | 1            |

We apply this dummification both to our training and to our test data. A nice side effect of that is that we do not have to deal with missing categorical values – they are simply set 0 in all the dummy columns (as shown in house 3 of the above example).

In [66]:
def dummify(data, update_cat_vars=False):
    # This allows us to alter the global variable CAT_VARS within the function.
    global CAT_VARS
    
    # First, we save the data into two new DataFrames, split into categorical and
    # continous features.
    cont_df = data[CONT_VARS]
    cat_df = data[CAT_VARS]
    cat_vars_new = list(CAT_VARS)
    
    # We iterate over each categorical variable, calculate the dummy variables,
    # insert them into the cat_df DataFrame, and, finally, delete the original
    # (categorical) feature from cat_df.
    # Additionally, we save the labels of our newly created dummy features in
    # CAT_VARS_new.
    for cat_var in CAT_VARS:
        dummies = pd.get_dummies(data[cat_var], prefix=cat_var)
        cat_df = cat_df.join(dummies)
        del cat_df[cat_var]
        
        cat_vars_new.remove(cat_var)
        cat_vars_new = cat_vars_new + dummies.columns.values.tolist()
        
    # This merges the continuous and categorical features back into one
    # DataFrame *result_df*.
    result_df = cat_df.join(cont_df)
    
    # Up to this point, the SalePrice is missing in our newly created DataFrame
    # result_df. Here we try to insert it again. This might fail because there
    # actually is no target variable in our test set (only in the train set). So,
    # if adding the (potentially missing) SalePrice fails, we just go on without
    # adding it.
    try:
        result_df = result_df.join(data[TARGET_VAR])
    except:
        pass
    
    # Only update the global CAT_VARS labels if we passed the argument
    # update_cat_vars to the function.
    if (update_cat_vars):
        CAT_VARS = cat_vars_new

    return result_df

# Finally, we dummify both train_df and test_df and print their shape to see how
# the number of columns has increased. When running the function with the train
# data we tell it to update our newly created dummy feature labels in the CAT_VARS
# variable.
test_df = dummify(test_df)
train_df = dummify(train_df, update_cat_vars=True)

print 'train:', train_df.shape
print 'test:', test_df.shape

train: (1460, 314)
test: (1459, 296)


As our columns have changed during dummifying the data, we store the altered list of feature labels in the train data in a new list. As our target variable is not a feature we use for prediction, we exclude it from the list.

In [67]:
VARS_X = list(train_df.columns)
VARS_X.remove(TARGET_VAR[0])

We dummified our test and train data separately. As not every class (category value) used in the train data is also used in the test data, and vice versa, our columns might no longer match. In order to make sure to realign the number of features (i.e. get the same number of columns), we check for columns which are present in the train data but missing in the test data, add these missing columns to the test data and fill them with zeros (as they apparently aren't present in the test set). Columns present in the test set but not in the train set are entirely dropped from the test set as our model cannot use them for prediction anyway.

In [68]:
cols_missing_in_test = set(VARS_X) - set(test_df.columns)
for col in cols_missing_in_test:
    test_df[col] = 0
    
cols_missing_in_train = set(test_df.columns) - set(VARS_X)
for col in cols_missing_in_train:
    del test_df[col]
    CAT_VARS = list(set(CAT_VARS) - set(cols_missing_in_train))

print 'train:', train_df.shape
print 'test:', test_df.shape

train: (1460, 314)
test: (1459, 313)


<font color="red">**TO DO:**</font> As outliers potentially bias our model (depending on the prediction model), they might need to be eliminated from the train data. In the following example we simply drop all the lines that differ in at least one column by more than 3 standard deviations from the column mean. But this seems to be a bad idea as it just makes our dataset a lot smaller. We get better mean squared errors when slicing our train data into a subset + validation set. But as soon as we apply it to the test data, Kaggle reports lower scores – so we are sort of punished for training or model based on a smaller train dataset. So we might need to investigate more sophisticated methods of outlier handling.

In [691]:
# Drop each row which has an outlier in at least one cell. An outlier is
# defined as being more than 3 standard devidations above or below the
# column mean.

# WARNING: This simple approach does not really work well as it drops nearly
#          all instances. A quick research revealed that we should either use
#          a more sophisticated outlier detection model (e.g. k-nearest neighbor
#          clustering), or that we should use a prediction model more robust to
#          outliers (e.g. Random Forest).


#for cont_var in CONT_VARS:
#    train_df = train_df[np.abs(train_df[cont_var] - train_df[cont_var].mean()) <= (3 * train_df[cont_var].std())]

#train_df = eliminate_outliers(train_df)
#print 'train:', train_df.shape

Both the train and the test data have a lot of missing values. The categorical variables are already covered but we still need to fill the gaps of the continuous features. The following fills all missing values by the entire column's mean. Note that the values used to fill the missing test data cells need to be based on the train data's column means.

In [69]:
train_df = train_df.fillna(train_df.mean())
test_df = test_df.fillna(train_df.mean())

The scales of our features vary to a great extent. Whereas the categorical features range from 0 to 1, continuous features like *BsmtUnfSF* range from 0 to well over a thousand. Some prediction models require features within the same scale. One way to achieve this is standardizing the data using z-cores. It's a convention which recalculates a column so that the mean equals 0 and one standard deviation equals 1. This way, the data is distributed more or less around 0. The scaler is learned based on the train data and subsequently applied to the train data. As our binary category variables already are within the desired scale, we only apply the standardization to the continous variables (as suggested by <a href="http://andrewgelman.com/2009/07/11/when_to_standar/" target="_blank">Gelman, 2009</a>)

In [693]:
# WARNING: The following code might not work as expected. It created some
#          weird negative results with the Linear Regression model. So better
#          use it with caution.

#scaler = preprocessing.StandardScaler().fit(train_df[CONT_VARS])
#train_df[CONT_VARS] = scaler.transform(train_df[CONT_VARS])
#test_df[CONT_VARS] = scaler.transform(test_df[CONT_VARS])

In order to get a sense of how the data looks at this point, we export our train and test data as csv files. After running the next code block, they can be found in the same folder as this notebook. Note that you might encounter errors if you try to export the files while still having an older version opened.

In [70]:
train_df.to_csv('clean_train.csv', sept=',', index=False)
test_df.to_csv('clean_test.csv', sept=',', index=False)

Last not least, we split our train data into two smaller fractions: into a train and a validation set. This allows us two measure the performance of our predictions (without involving the test set which we can only evaluate by uploading it to Kaggle).

In [84]:
# Store all the feature labels of train_df into a list; remove the SalePrice.
features = train_df.columns.tolist()
features.remove(TARGET_VAR[0])

# Generate the training set. Set random_state to be able to replicate results.
# Our train data will contain 80% of train_df.
train = train_df.sample(frac=0.8, random_state=1)

# Select anything not in the training set (20%) and put it in the validation set.
validation = train_df.loc[~train_df.index.isin(train.index)]

print "train (80%):"
print train.shape
print "validation (20%):"
print validation.shape

y = train.SalePrice

train (80%):
(1168, 314)
validation (20%):
(292, 314)


In [85]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=8)
neigh.fit(train[features], train[TARGET_VAR[0]])

KNN_prediction = (neigh.predict(validation[features]))
print KNN_prediction
print(neigh.predict_proba(validation[features]))

print metrics.accuracy_score(validation.SalePrice, KNN_prediction)
                                       
print 'Mean Squared Error:', mean_squared_error(KNN_prediction, validation.SalePrice)

[ 79900  90000 127000 110000 110000 110000 107000 167000 135000  55993
 105000 112000 165500 165500 203000 132000 147000 192500 173000 163500
 167000 167000 130000 100000 135000 184000 105000 119000  80000 145000
 150900 159000  87500 161500 167000 159000 110000 109000  82500 159895
 131000 159895 118000 188000 135000 163990 124000 105000 108000 107000
 240000 136900 172785 160000 187500  87000 155000 122500  82500 110000
 129900 119000  55993 160000 135000  76000 165500  91000  76000 125500
 175000 129000 260000  66500  52000 268000 193000  73000 160000  79000
  79000 101000 159895 136905 146500  85000 129000 253293 128000  75500
 136000 106000 105000 128000 163500 135000 111250  84500 121500 139000
 187500 137000 187500 122500 123000 137500  90000  61000 239686 280000
 108000  82500 109000 160000 102000  68400 250000  95000 104900 143000
 108000 144000  66500 210000 157000 128500 125500  97500 141000  79000
  79000 176000 131500 137000  60000 160000 129900 187500 115000 124500
  8200

# Linear Regression
Now we're finally ready two use our preprocessed data for training a linear regression model based on the 80 % train set.

In [86]:
# Initialize the model class.
linear_regression_model = LinearRegression() #(normalize=True)?

# Fit the model to the 80% training data.
linear_regression_model.fit(train[features], train[TARGET_VAR[0]])

# Generate our predictions for the validation set.
predictions = linear_regression_model.predict(validation[features])

predictions = predictions.astype(int)

# Compute error between our validation predictions and the actual values.
print 'Mean Squared Error:', mean_squared_error(predictions, validation[TARGET_VAR[0]])


Mean Squared Error: 507651587.144


Happy with the mean squared error (the lower the better)? If yes, we can improve our model by using the entire 100% of our train_df.

In [87]:
linear_regression_model = LinearRegression() #(normalize=True)?
linear_regression_model.fit(train_df[features], train_df[TARGET_VAR[0]])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

# Adding Learning curve

In [105]:
def learning_curve(X_train, y_train, X_test, y_test):
    """Calculate the performance of the model after a set of training data."""
    
    # We will vary the training set size so that we have 20 different sizes
    sizes = np.round(np.linspace(1, len(X_train), 20))
    train_err = np.zeros(len(sizes))
    test_err = np.zeros(len(sizes))
 
    for i, s in enumerate(sizes):
        # Create and fit the regressor model
        
        linear_regression_model.fit(X_train[:int(s)], y_train[:int(s)])
        linear_regression_model.fit(X_test[:int(s)], y_test[:int(s)])
        #regressor.fit(X_train[:s], y_train[:s])
   
        # Find the performance on the training and testing set
        train_err[i] = mean_squared_error(y_train[:int(s)], linear_regression_model.predict(X_train[:int(s)]))
        test_err[i] = mean_squared_error(y_test, linear_regression_model.predict(X_test))
        
    # Plot learning curve graph
    learning_curve_graph(sizes, train_err, test_err)

def learning_curve_graph(sizes, train_err, test_err):
    """Plot training and test error as a function of the training size."""

    plt.figure()
    plt.title('LinearRegression: Performance vs Training Size')
    plt.plot(sizes, test_err, lw=2, label = 'test error')
    plt.plot(sizes, train_err, lw=2, label = 'training error')
    plt.legend()
    plt.xlabel('Training Size')
    plt.ylabel('Error')
    plt.show()
    

In [106]:
learning_curve(train[features], train[TARGET_VAR[0]], validation[features], validation[TARGET_VAR[0]])
#learning_curve(X_train, y_train, X_test, y_test)

Now we can use this improved model to predict the housing prices of our test data.

In [40]:
predictions = linear_regression_model.predict(test_df[features])
predictions.shape

(1459L,)

Due to a lack of a validation set, we are now no longer able to compute the mean squared error. But by uploading our predictions for the test set to Kaggle, we can get an even better sense of how good we're doing. This requires preparing a csv file according to Kaggle's requirements: one column with the ID, and one column with the predicted SalePrice. We create a function we can also reuse for other models than Linear Regression.

In [750]:
def export_csv(predictions, model):
    # Get the IDs from the indeces we earlier stored in test_df.
    ids = test_df.index.values

    # Stack the IDs and predictions into a numpy array and transpose it into
    # vertical form.
    submission = np.vstack((ids, predictions)).T

    # Convert the submission array into a Pandas DataFrame object.
    submission = pd.DataFrame(data=submission, columns=['Id', 'SalePrice'])

    # Convert Id from float to int to avoid .0 notation.
    submission['Id'] = submission['Id'].astype(int)
    
    # Convert possible negative predicted values to 0
    submission[submission < 0] = 0

    # Print the shape of the newly created submission DataFrame.
    print 'Submission:', submission.shape
    
    # Export the submissions to a csv file.
    submission.to_csv('submission_' + model + '.csv', sept=',', index=False)

Now we can call the export_csv function to print our Linear Regression predictions to a csv file called submission_linear_regression.csv.

In [751]:
export_csv(predictions, 'linear_regression')

Submission: (1459, 2)


# Ridge Regression

In [767]:
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
#alphas = [0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64, 1.28, 2.56, 5.12, 10.24, 20.48, 40.96]

mean_squared_errors = np.zeros((len(alphas), 2))

for i, alph in enumerate(alphas):
    ridge_regression_model = Ridge(alpha = alph)
    ridge_regression_model.fit(train[features], train[TARGET_VAR[0]])
    predictions = ridge_regression_model.predict(validation[features])
    
    mean_squared_errors[i] = [alph, mean_squared_error(predictions, validation[TARGET_VAR[0]])]

ridge_regression_model = Ridge(alpha = 0.08)
ridge_regression_model.fit(train_df[features], train_df[TARGET_VAR[0]])
predictions = ridge_regression_model.predict(test_df[features])

export_csv(predictions, 'ridge_regression')

Submission: (1459, 2)


# Create graph

In [768]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

#alphas = [0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64, 1.28, 2.56, 5.12, 10.24, 20.48, 40.96]
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation - Just Do It")
pl.xlabel("alpha")
pl.ylabel("rmse")
pl.show()

In [762]:
cv_ridge.min()

6.1881357016154804e-09

# Random Forest Regressor
Now we can try out another method: Random Forest Regressor. It works pretty much the same like the Linear Regression model.

In [610]:
# Initialize the model.
# Note: The parameters given here are not really optimized yet. Apparantly, n_estimator
#       is the one we should pay the most attention to. For details see:
#       http://scikit-learn.org/stable/modules/ensemble.html#parameters
random_forest_model = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=1)

# Fit the model to the 80% train data.
random_forest_model.fit(train[features], train[TARGET_VAR[0]])

# Make predictions for our validation set.
predictions = random_forest_model.predict(validation[features])

# Compute the error.
print 'Mean Sqared Error:', mean_squared_error(predictions, validation[TARGET_VAR[0]])

Mean Sqared Error: 639153795.737


Again, when we are happy with the Mean Squared Error, we can fit the model to our 100% train_df dataset, predict the values for Kaggle's test dataset, and export the results into a file called submission_random_forest_regressor.csv.

In [608]:
# TO DO: Optimize parameters (see comments in the codeblock above).
random_forest_model = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=1)
random_forest_model.fit(train_df[features], train_df[TARGET_VAR[0]])

predictions = random_forest_model.predict(test_df[features])

In [609]:
export_csv(predictions, 'random_forest_regressor')

Submission: (1459, 2)
